In [52]:
video_file = "takeout_folders/takeout-20230525T041716Z-001/Takeout/Google Photos/Photos from 2015/IMG_0290.JPG"


In [53]:
from exiftool import ExifToolHelper
import json
from datetime import datetime
import shutil
import math

In [54]:
with open(f'{video_file}.json') as json_file:
    metadata = json.load(json_file)

In [55]:
metadata

{'title': 'IMG_0290.JPG',
 'description': '',
 'imageViews': '65',
 'creationTime': {'timestamp': '1473126679',
  'formatted': 'Sep 6, 2016, 1:51:19â€¯AM UTC'},
 'photoTakenTime': {'timestamp': '1438460509',
  'formatted': 'Aug 1, 2015, 8:21:49â€¯PM UTC'},
 'geoData': {'latitude': 40.3828611,
  'longitude': -105.5190111,
  'altitude': 2336.0,
  'latitudeSpan': 0.0,
  'longitudeSpan': 0.0},
 'geoDataExif': {'latitude': 40.3828611,
  'longitude': -105.5190111,
  'altitude': 2336.0,
  'latitudeSpan': 0.0,
  'longitudeSpan': 0.0},
 'url': 'https://lh3.googleusercontent.com/KBziBLdc5gDW7Oy8WUq8VAJZaMj2G1Tq1END3CUGvGkUV3Z-13Z2T3DowD4-b3ZbeOW_4UTRdsvvpmAEwW28S0ULrfOuGdM5zGDM1Qo',
 'googlePhotosOrigin': {'mobileUpload': {'deviceType': 'IOS_PHONE'}}}

In [56]:
 # Set timestmap metadata
timestamp = datetime.utcfromtimestamp(int(metadata['photoTakenTime']['timestamp'])).strftime("%Y:%m:%d %H:%M:%S")


In [57]:
shutil.copyfile(video_file, "output/test.mp4")

'output/test.mp4'

In [65]:
# https://stackoverflow.com/a/52371976/3675086
def deg_to_dms(deg, kind='lat'):
    decimals, number = math.modf(deg)
    d = int(number)
    m = int(decimals * 60)
    s = (deg - d - m / 60) * 3600.00
    compass = {
        'lat': ('N','S'),
        'lon': ('E','W')
    }
    compass_str = compass[kind][0 if d >= 0 else 1]
    return {
        'dms': (abs(d), abs(m), abs(s)),
        'ref': compass_str
    }

In [68]:
# convert decimal to degree, minute, second
lat = deg_to_dms(metadata['geoData']['latitude'])
lon = deg_to_dms(metadata['geoData']['longitude'], kind='lon')

In [69]:
lat, lon

({'dms': (40, 22, 58.29995999999833), 'ref': 'N'},
 {'dms': (105, 31, 8.439960000000424), 'ref': 'W'})

In [61]:
with ExifToolHelper() as et:
    et.set_tags(
        [video_file],
        tags={
            "DateTimeOriginal": timestamp,
            "GPSLatitude": str(metadata['geoData']['latitude']),
            "GPSLatitudeRef": lat['ref'],
            "GPSLongitude": str(metadata['geoData']['longitude']),
            "GPSLongitudeRef": lon['ref'],
        },
        params=["-o", "output/test.jpg"]
    )

In [62]:
with ExifToolHelper() as et:
    for d in et.get_tags(["output/test.jpg"], tags=["DateTimeOriginal", "GPSLatitude", "GPSLongitude"]):
        for k, v in d.items():
            print(f"Dict: {k} = {v}")

Dict: SourceFile = output/test.jpg
Dict: EXIF:DateTimeOriginal = 2015:08:01 20:21:49
Dict: EXIF:GPSLatitude = 40.3828610999861
Dict: Composite:GPSLatitude = 40.3828610999861
Dict: EXIF:GPSLongitude = 105.519011099993
Dict: Composite:GPSLongitude = 105.519011099993
